<a href="https://colab.research.google.com/github/Ashrithajangam/IT7133-FALL24/blob/main/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 930.1 kB/s eta 0:00:00


# **Uploading the Dataset**

In [3]:
from google.colab import files
import pandas as pd

uploaded = files.upload()
data = pd.read_csv(next(iter(uploaded.keys())))
display(data)

Saving credit_loan_approval.csv to credit_loan_approval (1).csv


,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,22,59000,RENT,123,PERSONAL,D,35000,16.02,0.59,Y,3,approved
1,21,9600,OWN,5,EDUCATION,B,1000,11.14,0.10,N,2,denied
2,25,9600,MORTGAGE,1,MEDICAL,C,5500,12.87,0.57,N,3,approved
3,23,65500,RENT,4,MEDICAL,C,35000,15.23,0.53,N,2,approved
4,24,54400,RENT,8,MEDICAL,C,35000,14.27,0.55,Y,4,approved
...,...,...,...,...,...,...,...,...,...,...,...,...
28633,57,53000,MORTGAGE,1,PERSONAL,C,5800,13.16,0.11,N,30,denied
28634,54,120000,MORTGAGE,4,PERSONAL,A,17625,7.49,0.15,N,19,denied
28635,65,76000,RENT,3,HOMEIMPROVEMENT,B,35000,10.99,0.46,N,28,approved
28636,56,150000,MORTGAGE,5,PERSONAL,B,15000,11.48,0.10,N,26,denied


# **Data Splitting**

In [6]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# **Numeric and Categorical Columns**

In [10]:
# Define numeric columns
num_cols = [
    'person_age',             # Age of the person
    'person_income',          # Annual income of the person
    'person_emp_length',      # Employment length of the person in years
    'loan_amnt',              # Amount of the loan
    'loan_int_rate',          # Interest rate of the loan
    'loan_percent_income',    # Percentage of income the loan amount represents
    'cb_person_cred_hist_length'  # Length of credit history of the person
]
print("Numeric columns (used for quantitative features):")
display(pd.DataFrame(num_cols, columns=["Numeric Columns"]))

# Define categorical columns
cat_cols = [
    'person_home_ownership',  # Type of home ownership (e.g., RENT, OWN, MORTGAGE)
    'loan_intent',            # Purpose of the loan (e.g., MEDICAL, PERSONAL)
    'loan_grade',             # Credit grade of the loan
    'cb_person_default_on_file'  # Default history flag (e.g., Y or N)
]
print("Categorical columns (used for qualitative features):")
display(pd.DataFrame(cat_cols, columns=["Categorical Columns"]))

all_cols = num_cols + cat_cols

print("All columns (features used for modeling):")
display(pd.DataFrame(all_cols, columns=["All Columns"]))

target = 'loan_status'

train_data = train_data[all_cols + [target]]
val_data = val_data[all_cols + [target]]
test_data = test_data[all_cols + [target]]

print("Sample of Training Data:")
display(train_data.head())


Numeric columns (used for quantitative features):


,Numeric Columns
0,person_age
1,person_income
2,person_emp_length
3,loan_amnt
4,loan_int_rate
5,loan_percent_income
6,cb_person_cred_hist_length


Categorical columns (used for qualitative features):


,Categorical Columns
0,person_home_ownership
1,loan_intent
2,loan_grade
3,cb_person_default_on_file


All columns (features used for modeling):


,All Columns
0,person_age
1,person_income
2,person_emp_length
3,loan_amnt
4,loan_int_rate
5,loan_percent_income
6,cb_person_cred_hist_length
7,person_home_ownership
8,loan_intent
9,loan_grade


Sample of Training Data:


,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,person_home_ownership,loan_intent,loan_grade,cb_person_default_on_file,loan_status
1449,24,15120,4,1600,7.51,0.11,3,RENT,MEDICAL,A,N,denied
3935,22,75000,0,4800,12.21,0.06,2,RENT,DEBTCONSOLIDATION,B,N,denied
27727,41,102000,20,7500,11.49,0.07,17,MORTGAGE,VENTURE,B,N,denied
27307,44,67000,3,10000,11.66,0.15,15,RENT,DEBTCONSOLIDATION,C,Y,denied
5807,22,54996,4,6000,10.75,0.11,3,RENT,VENTURE,B,N,denied


# **Encoding Categorical Variables**

In [11]:
from sklearn.preprocessing import LabelEncoder

cat_dims = {}
for col in cat_cols:
    label_encoder = LabelEncoder()
    train_data[col] = label_encoder.fit_transform(train_data[col].values)
    val_data[col] = label_encoder.transform(val_data[col].values)
    test_data[col] = label_encoder.transform(test_data[col].values)
    cat_dims[col] = len(label_encoder.classes_)

# **Model Preparation**

In [12]:
cat_idxs = [i for i, f in enumerate(all_cols) if f in cat_cols]
cat_dims = [cat_dims[f] for i, f in enumerate(all_cols) if f in cat_cols]

#model parameters
n_decisions = 64
n_attentions = 64
n_steps = 5
learning_rate = 0.01

from pytorch_tabnet.tab_model import TabNetClassifier
import torch

#TabNet model
tabnet = TabNetClassifier(
    n_d=n_decisions,
    n_a=n_attentions,
    n_steps=n_steps,
    cat_dims=cat_dims,
    cat_emb_dim=cat_dims,
    cat_idxs=cat_idxs,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=learning_rate)
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


# **Training the Model**

In [13]:
n_epochs = 500
batch_size = 1024
patience = 20

X_train = train_data[all_cols].values
y_train = train_data[target].values
X_val = val_data[all_cols].values
y_val = val_data[target].values

tabnet.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_name=['train', 'valid'],
    max_epochs=n_epochs,
    patience=patience,
    batch_size=batch_size
)

epoch 0  | loss: 0.59158 | train_auc: 0.59796 | valid_auc: 0.60353 |  0:00:06s
epoch 1  | loss: 0.43554 | train_auc: 0.63788 | valid_auc: 0.61443 |  0:00:14s
epoch 2  | loss: 0.39484 | train_auc: 0.65259 | valid_auc: 0.66969 |  0:00:20s
epoch 3  | loss: 0.35881 | train_auc: 0.7169  | valid_auc: 0.7048  |  0:00:26s
epoch 4  | loss: 0.34748 | train_auc: 0.70509 | valid_auc: 0.67739 |  0:00:32s
epoch 5  | loss: 0.33402 | train_auc: 0.71774 | valid_auc: 0.70141 |  0:00:38s
epoch 6  | loss: 0.32903 | train_auc: 0.71317 | valid_auc: 0.70316 |  0:00:44s
epoch 7  | loss: 0.3092  | train_auc: 0.70266 | valid_auc: 0.67105 |  0:01:00s
epoch 8  | loss: 0.31057 | train_auc: 0.74226 | valid_auc: 0.72224 |  0:01:15s
epoch 9  | loss: 0.30498 | train_auc: 0.75985 | valid_auc: 0.74071 |  0:01:23s
epoch 10 | loss: 0.30188 | train_auc: 0.8045  | valid_auc: 0.79398 |  0:01:29s
epoch 11 | loss: 0.30157 | train_auc: 0.77984 | valid_auc: 0.77019 |  0:01:35s
epoch 12 | loss: 0.30184 | train_auc: 0.8358  | vali

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


# **Testing and Evaluation**

In [15]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

#Predict probabilities and display in tabular format
test_probs = tabnet.predict_proba(test_data[all_cols].values)
display(pd.DataFrame(test_probs, columns=[f'Class_{i}' for i in range(test_probs.shape[1])]))

#Predict class indices and display
test_pred_id = np.argmax(test_probs, axis=1)
display(pd.DataFrame(test_pred_id, columns=['Predicted Class Index']))

#Map indices to class labels and display
id2label = {idx: label for idx, label in enumerate(tabnet.classes_)}
test_pred_label = np.array([id2label[x] for x in test_pred_id])
display(pd.DataFrame(test_pred_label, columns=['Predicted Class Label']))

#Compare actual vs. predicted results and display
results_df = pd.DataFrame({
    'Actual': test_data[target].values,
    'Predicted': test_pred_label,
    'Predicted Class Index': test_pred_id
})
display(results_df.head(10))

#Displaying accuracy and F1 score
accuracy = accuracy_score(test_data[target], test_pred_label)
f1 = f1_score(test_data[target], test_pred_label, average='weighted')
print('Test Accuracy:', accuracy)
print('Test F1 Score:', f1)


,Class_0,Class_1
0,0.038415,0.961585
1,0.999545,0.000455
2,0.008446,0.991554
3,0.123229,0.876771
4,0.769232,0.230768
...,...,...
5723,0.004556,0.995444
5724,0.562014,0.437986
5725,0.085354,0.914646
5726,0.065375,0.934625


,Predicted Class Index
0,1
1,0
2,1
3,1
4,0
...,...
5723,1
5724,0
5725,1
5726,1


,Predicted Class Label
0,denied
1,approved
2,denied
3,denied
4,approved
...,...
5723,denied
5724,approved
5725,denied
5726,denied


,Actual,Predicted,Predicted Class Index
0,denied,denied,1
1,approved,approved,0
2,denied,denied,1
3,approved,denied,1
4,approved,approved,0
5,denied,denied,1
6,denied,denied,1
7,denied,denied,1
8,approved,denied,1
9,denied,denied,1


Test Accuracy: 0.9079958100558659
Test F1 Score: 0.8994947452732295
